# Setup

In [ ]:
%run CvCmdApi.py

CvCmder = CvCmdHandler()

# Demo

In [ ]:
startPrint = False
while True:
    flags = CvCmder.CvCmd_Heartbeat(300, 400, 30.12, 40.13)
    if startPrint:
        print(flags)
        time.sleep(1)
        # if True not in flags:
        #     startPrint = False
    elif True in flags:
        startPrint = True


In [ ]:
CvCmder.ser.close()

In [ ]:
CvCmder.CvCmd_Reset()

## Unit test

In [ ]:
#AutoAim_Heartbeat(10,10,1)
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
CvCmder.ser.write(CvCmder.txAckMsg)

In [ ]:
(gimbal_coordinate_x, gimbal_coordinate_y, chassis_speed_x, chassis_speed_y) = (300, 400, 30.12, 40.13)
CvCmder.txCvCmdMsg[CvCmder.DATA_PAYLOAD_INDEX:CvCmder.DATA_PAYLOAD_INDEX+12] = b''.join([gimbal_coordinate_x.to_bytes(2, 'little'), gimbal_coordinate_y.to_bytes(2, 'little'), struct.pack('<f', chassis_speed_x), struct.pack('<f', chassis_speed_y)])
CvCmder.ser.write(CvCmder.txCvCmdMsg)

In [ ]:
# incomplete/invalid msg
CvCmder.ser.write(CvCmder.txAckMsg[:10])
CvCmder.ser.write(CvCmder.txAckMsg[5:])